In [ ]:
from google.colab import userdata
API_KEY = userdata.get('GoogleAI_API_KEY')

In [ ]:
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
genai.configure(api_key=API_KEY)
gemini = genai.GenerativeModel(model_name="models/gemini-2.0-flash-exp")

In [ ]:
try:
    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp", google_api_key=API_KEY)
except Exception as e:
    print(e)

In [ ]:
res = gemini.generate_content('who are you?')

In [ ]:
print(res)

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader
loader = CSVLoader("dataset.csv")

data = loader.load()

In [ ]:
data[:5]

In [ ]:
for doc in data[:5]:  # Check the first 5 entries
    print(doc.page_content)

Question: What is the best care for your skin?
Answer: "The best skin care involves following a concrete skincare routine, healthy dietary habits, and a proper sleep cycle. Our facial skin deals with pollution, climate fluctuations, the suns UV rays, and poor-quality facial accessories. Thus, consistency in the skincare routine plays a key role in improving skin health. In addition, cutting off stress, smoking habits, and junk foods from life contribute hugely to overall skin health. You should always opt for gentle skin care products with natural and toxin-free ingredients."
Question: Are Mamaearth skin care products natural?
Answer: "Yes. Mamaearth skincare products do not include harmful chemicals to hurt any skin barrier. Besides, it is the first Asian brand to get the approval of MadeSafe, meaning its product range is free of any harmful chemicals that may damage the skin or cause allergies. So, if you are keen on using the best skin care products with mild and safe ingredients t

In [ ]:
from sentence_transformers import SentenceTransformer

# Load the embedding model
embedder = SentenceTransformer('all-MiniLM-L6-v2')

texts = [doc.page_content for doc in data]

# Generate embeddings
embeddings = embedder.encode(texts, convert_to_tensor=False)  # FAISS works with numpy arrays
print(f"Embeddings shape: {embeddings.shape}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embeddings shape: (242, 384)


In [ ]:
embeddings[:4]

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings

# Create a FAISS vector store from your documents and embeddings
# Use SentenceTransformerEmbeddings to wrap your SentenceTransformer model
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vector_store = FAISS.from_documents(
    documents=data,
    embedding=embeddings
)

# Save the index (optional, for reuse)
vector_store.save_local("faiss_index")

In [ ]:
print(llm.invoke("Hello, are you working?"))

In [ ]:
from langchain.chains import RetrievalQA

# Set up the retriever
retriever = vector_store.as_retriever(search_kwargs={"k": 3})  # Retrieve top 3 matches

# Create a RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # Combines retrieved docs into the prompt
    retriever=retriever,
    return_source_documents=True  # Optional: see what was retrieved
)

# Test it
query = "what best products can men use and suggest me products to handle acne scars?"
result = qa_chain({"query": query})
print(result["result"])
print(result["source_documents"])

For the best skin care for men, it is suggested to go for a gentle face cleanser with deep-cleansing formula and a moisturizer with an oil-free yet hydrating formula. Applying SPF is non-negotiable, so find a lightweight sunscreen with SPF 50 for better protection. Regular exfoliation is necessary to keep pores clean and healthy. So, add a nourishing face scrub and facial mask to your weekly routine. Gradually, you may introduce toners and treatment products such as serums and ointments to your routine if you have any particular concerns.

Mamaearth offers various face care products formulated specially to treat acne scars, blemishes, and hyperpigmentation. You may try skin care products featuring Tea Tree, Vitamin C, Aloe Vera, and Rice Water for the best results. It is necessary to be consistent with cleansing and exfoliation while dealing with acne scars. The sunblock is a crucial thing to stick with for effective results. In addition, you may also consider using Mamaearths Retinol